## System setup

---


##### Importing modules


In [1]:
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import (
    TensorBoard,
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
)

import keras_tuner

from toolkit.classes import DataSet, ImageProcessor, Augmenter, Model

from tensorflow.keras.layers import BatchNormalization

2023-01-17 18:06:38.910488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 18:06:40.171607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 18:06:40.171679: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-17 18:06:45.961166: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Read dataset

---


##### Create image dataframe


In [2]:
images = DataSet()


In [3]:
datasets = ["raw", "data", "augmented", "preprocessed"]
for dataset in datasets:
    images.update_dataset(dataset=dataset)

Found 150 images in ./raw/Train/Non defective/
Found 150 images in ./raw/Train/Defective/
Found 31 images in ./raw/Validation/Non defective/
Found 31 images in ./raw/Validation/Defective/
Found 11 images in ./raw/Test/Non defective/
Found 11 images in ./raw/Test/Defective/
Found 150 images in ./data/Train/Non defective/
Found 150 images in ./data/Train/Defective/
Found 31 images in ./data/Validation/Non defective/
Found 31 images in ./data/Validation/Defective/
Found 11 images in ./data/Test/Non defective/
Found 11 images in ./data/Test/Defective/
Found 25 images in ./augmented/Train/Non defective/
Found 25 images in ./augmented/Train/Defective/
Found 175 images in ./preprocessed/Train/Non defective/
Found 175 images in ./preprocessed/Train/Defective/
Found 31 images in ./preprocessed/Validation/Non defective/
Found 31 images in ./preprocessed/Validation/Defective/
Found 11 images in ./preprocessed/Test/Non defective/
Found 11 images in ./preprocessed/Test/Defective/


## Hypertuner

---


##### Hypertune Learning Rate


In [4]:
CNN = Model("VGG16")
CNN.callbacks = [
    TensorBoard(log_dir="./tuner/logs"),
]
CNN.epochs = 20
CNN.load_datasets(color_mode='rgb')

def hp_model(hp):
    CNN.build_model()
    CNN.optimizer = Adam(
        learning_rate=hp.Float("lr", min_value=1e-6, max_value=1e-3, sampling="log")
    )
    CNN.compile_model()
    return CNN.model

hp_model(keras_tuner.HyperParameters())
tuner = keras_tuner.RandomSearch(
    hypermodel=hp_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="./tuner/",
    project_name=f"{CNN.name}_LR",
)
tuner.search(
    CNN.data["Train"],
    validation_data=CNN.data["Validation"],
    epochs=CNN.epochs,
    callbacks=CNN.callbacks,
)

Trial 10 Complete [03h 26m 52s]
val_accuracy: 0.6774193644523621

Best val_accuracy So Far: 0.774193525314331
Total elapsed time: 1d 10h 26m 32s
INFO:tensorflow:Oracle triggered exit


In [5]:
tuner.results_summary()

Results summary
Results in ./tuner/VGG16_LR
Showing 10 best trials
Trial summary
Hyperparameters:
lr: 1.1448657686406566e-05
Score: 0.774193525314331
Trial summary
Hyperparameters:
lr: 2.9808534935197715e-06
Score: 0.7419354915618896
Trial summary
Hyperparameters:
lr: 7.48258459362723e-06
Score: 0.7419354915618896
Trial summary
Hyperparameters:
lr: 2.736866531395312e-06
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 0.00010809378085025995
Score: 0.6935483813285828
Trial summary
Hyperparameters:
lr: 1.0755850754957174e-06
Score: 0.6774193644523621
Trial summary
Hyperparameters:
lr: 1.5513379998103392e-06
Score: 0.6290322542190552
Trial summary
Hyperparameters:
lr: 9.57678040771624e-05
Score: 0.5
Trial summary
Hyperparameters:
lr: 0.00026932531091637506
Score: 0.5
Trial summary
Hyperparameters:
lr: 0.00037178483328028776
Score: 0.5


In [10]:
CNN = Model("ResNet50_pretrained")
CNN.callbacks = [
    TensorBoard(log_dir="./tuner/logs"),
]
CNN.epochs = 10
CNN.load_datasets(color_mode='rgb')

def hp_model(hp):
    CNN.build_model()
    CNN.model.load_weights('./models/ResNet50_pretrained_LR.h5')
    CNN.pretrained_model.trainable = True
    for layer in CNN.pretrained_model.layers:
        layer.trainable = not isinstance(layer, BatchNormalization)
    CNN.optimizer = Adam(
        learning_rate=hp.Float("lr", min_value=1e-8, max_value=1e-6, sampling="log")
    )
    CNN.compile_model()
    return CNN.model

hp_model(keras_tuner.HyperParameters())
tuner = keras_tuner.RandomSearch(
    hypermodel=hp_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="./tuner/",
    project_name=f"{CNN.name}_LR",
)
tuner.search(
    CNN.data["Train"],
    validation_data=CNN.data["Validation"],
    epochs=CNN.epochs,
    callbacks=CNN.callbacks,
)

Trial 10 Complete [00h 41m 05s]
val_accuracy: 0.725806474685669

Best val_accuracy So Far: 0.7419354915618896
Total elapsed time: 06h 51m 11s
INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()


Results summary
Results in ./tuner/ResNet50_pretrained_LR
Showing 10 best trials
Trial summary
Hyperparameters:
lr: 2.2000693929233267e-08
Score: 0.7419354915618896
Trial summary
Hyperparameters:
lr: 6.948639754218684e-08
Score: 0.7419354915618896
Trial summary
Hyperparameters:
lr: 1.3894728980086928e-08
Score: 0.7419354915618896
Trial summary
Hyperparameters:
lr: 3.448238582327038e-07
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 1.7224708407553915e-07
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 4.946049376824357e-08
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 5.0177763558943696e-08
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 7.485507695241471e-07
Score: 0.725806474685669
Trial summary
Hyperparameters:
lr: 5.76897893744754e-07
Score: 0.7096773982048035
Trial summary
Hyperparameters:
lr: 3.7757747930199216e-07
Score: 0.7096773982048035
